In [ ]:
import os


# Utility to remove duplicate annotation sets from a SemEval 2021 - MeasEval submission

In [ ]:
# Replace these three variables with your appropriate values

# Directory containing MeasEval submissions
submissions = ""

# subdirectory containing the potentially duplicate TSV annotation sets.
inputSub = "tsv"

# subdirectory to write the de-duplicated TSV annotation sets.
outputSub = "gpt3-tsv-dedupe"

In [2]:
# Initialize work variables
subsWithExact = 0
subsWithSet = 0
subNamesWithSet = []
subsWithOverlap = 0
subsWithContain = 0
totalSubs = 0

# Process all the files in the 
for sub in os.listdir(submissions):
    # This could be flattened, but I have as a loop so I can run the stats on all submissions
    if sub == inputSub:
        exactDupCount = 0
        setDupCount = 0
        overlapDupCount = 0
        containedInCount = 0
        uniqCount = 0
        totalCount = 0
        totalSubs += 1
        for fn in os.listdir(submissions+sub+"/"):
            if fn != "__MACOSX":
                with open(submissions+sub+"/"+fn, errors = "ignore") as tsvfile:
                    next(tsvfile)
                    keys = []
                    fullkeys = []
                    offsets = []
                    skip = []
                    # Comment these out if not writing a new copy of the output.
                    tsv = open(submissions+outputSub+"/"+fn, "w")
                    tsv.write("docId\tannotSet\tannotType\tstartOffset\tendOffset\tannotId\ttext\tother\n")
                    tsvdata = tsvfile.read().splitlines()
                    for line in tsvdata:
                        if len(line.split("\t")) == 8:
                            docId = line.split("\t")[0]
                            annotSet = line.split("\t")[1]
                            atype = line.split("\t")[2]
                            start = line.split("\t")[3]
                            end = line.split("\t")[4]
                            text = line.split("\t")[6]
                            if atype == "Quantity":
                                key = "\t".join([docId, start, end, text])
                                fullkey = {atype: key}
                                subQuals = []
                                for subline in tsvdata:
                                    if len(subline.split("\t")) == 8 and subline.split("\t")[2] != "Quantity":
                                        subId = subline.split("\t")[0]
                                        subSet = subline.split("\t")[1]
                                        subType = subline.split("\t")[2]
                                        subStart = subline.split("\t")[3]
                                        subEnd = subline.split("\t")[4]
                                        subText = subline.split("\t")[6]
                                        subKey = "\t".join([subId, subStart, subEnd, subText])
                                        if (subSet == annotSet and subId == docId):
                                            if subType != "Qualifier":
                                                #print("SubKey: " + subKey)
                                                fullkey[subType] = subKey
                                            else:
                                                subQuals.append(subKey)
                                fullkey["Qualifier"] = subQuals
                                if fullkey in fullkeys:
                                    setDupCount+=1
                                    # Move this skip.append(annotSet) to whichever block we're omitting
                                    skip.append(annotSet)
                                elif key in keys:
                                    exactDupCount+=1
                                else:
                                    overlapMatches = 0
                                    containedMatches = 0
                                    for offset in offsets:
                                        if ((docId == offset[0] and int(start) >= int(offset[1]) and int(start) < int(offset[2])) or
                                            (docId == offset[0] and int(end) > int(offset[1]) and int(end) <= int(offset[2]))):
                                            overlapMatches+=1
                                        elif ((docId == offset[0] and int(start) >= int(offset[1]) and int(start) <= int(offset[2]) and
                                            docId == offset[0] and int(end) >= int(offset[1]) and int(end) <= int(offset[2])) or
                                            (docId == offset[0] and int(offset[1]) >= int(start) and int(offset[1]) <= int(end) and
                                            docId == offset[0] and int(offset[2]) >= int(start) and int(offset[2]) <= int(end))):
                                            containedMatches+=1
                                    if overlapMatches > 0:
                                        overlapDupCount+=1
                                    elif containedMatches > 0:
                                        containedInCount+=1
                                    else:
                                        uniqCount+=1
                                fullkeys.append(fullkey)
                                keys.append(key)
                                offsets.append((docId, start, end))
                                totalCount +=1
                            # Comment these next lines if not writing new files
                            if annotSet not in skip:
                               tsv.write(line+"\n")
                    tsv.close()
        print(sub + " has Total Quantities: " + str(totalCount))
        print(sub + " has exact Quantity-only dups: " + str(exactDupCount))
        print(sub + " has full set dups: " + str(setDupCount))
        print(sub + " has quant overlap dups: " + str(overlapDupCount))
        print(sub + " has quant containedIn dups: " + str(containedInCount))
        print(sub + " has uniq Quantities : " + str(uniqCount))
        if exactDupCount > 0: 
            subsWithExact += 1
        if setDupCount > 0: 
            subsWithSet += 1
            subNamesWithSet.append(sub)
        if overlapDupCount > 0: 
            subsWithOverlap += 1
        if containedInCount > 0: 
            subsWithContain += 1
print("###")
print(str(subsWithExact) + " submissions have exact duplicates")
print(str(subsWithSet) + " submissions have set duplicates")
print(str(subsWithOverlap) + " submissions have overlapping duplicates")
print(str(subsWithContain) + " submissions have containedIn duplicates")
print("Out of " + str(totalSubs) + " total submissions")

gpt3 has Total Quantities: 496
gpt3 has exact Quantity-only dups: 21
gpt3 has full set dups: 169
gpt3 has quant overlap dups: 8
gpt3 has quant containedIn dups: 0
gpt3 has uniq Quantities : 298
###
1 submissions have exact duplicates
1 submissions have set duplicates
1 submissions have overlapping duplicates
0 submissions have containedIn duplicates
Out of 1 total submissions
